In [1]:
# @title INSTALL OPENAI
!pip install openai --upgrade
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3


In [ ]:
# @title LLAMA3
import json
import pandas as pd
import time
from openai import OpenAIError
import os
from openai import OpenAI

client = OpenAI(
    api_key="",
    base_url="https://api.together.xyz/v1",
    )

In [2]:
# @title OPENAI
import json
import pandas as pd
import time
import openai
from openai import OpenAIError

# Replace 'your_api_key_here' with your actual OpenAI API key
openai.api_key = ''


In [ ]:
completion = openai.chat.completions.create(
  model="gpt-4o-2024-08-06",
  temperature=0,
  max_tokens=2500,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=1,
  messages=[{"role":"system","content":"Act as an English grammar teacher. Your task will be:\n- Carefully analyze the answer to the question.\n- Check if answer is related to question (context is not important)\nINSTRUCTION\nSTEP 1: CHECK\n- Check if answer is grammar correct.\n- Check if there is any grammar mistake\n- Check if answer partly related the question.\n- Accept informal vocabulary usage as correct.\n- Accept user's answer is correct in every context\n + If answer check is true, do not give any suggestion.\n+ If check is false, give correct answer\n- Accept correct answer phrase, or extened answer as correct.\n- Accept inccorrect punctuation (. , ! ?...) mistakes as correct.\nSTEP 2: RESPOND:\n- give short, simple, brief explanation, and correct answer if false\n+ if answer correct, respond with:\n{\"explanation\":null,\"fixed_answer\":<correct answer>,\"correct_grammar\":\"yes\"}\n\n+ If answer incorrect, never let fixed_answer:null and respond with:\n{\"explanation\":<talk in Vietnamese, explain in detail why [answer] wrong, and correct grammar, then give example answer sentence>,\"fixed_answer\":<must give example correct answer, not null>,\"correct_grammar\":\"no\"}\n- remember to us ' instead of \" to quote text within JSON. 'I want to' instead of \"I want to'"},{"role":"user","content":"question: What is the most popular dish in your country?\nanswer: The most popular dish in my country is Pizza. You should try when you can!"},{"role":"assistant","content":"{\"explanation\":null,\"fixed_answer\":\"The most popular dish in my country is Pizza\",\"correct_grammar\":\"yes\"}"},{"role":"user","content":"question: How long have you been working there?\nanswer: I work there for 5 years."},{"role":"assistant","content":"{\"explanation\":\"Ở đây bạn cần sử dụng thì hiện tại hoàn thành tiếp diễn (present perfect continuous) để diễn tả thời gian bạn đã làm việc tại đó từ quá khứ đến hiện tại. Vì vậy câu trả lời đúng sẽ là 'I have been working there for 5 years.'\",\"fixed_answer\":\"I have been working there for 5 years.\",\"correct_grammar\":\"no\"}"},
  {"role":"user","content":"i need no grandma"}

  ]

)
response_content = completion.choices[0].message.content
print(response_content)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_IckUVKG7wcS1uCXP8Kpo25oH` does not have access to model `gpt-4o-2024-08-06`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
# @title OPEN AI CÓ MESSAGE HISTORY
def process_conversation(order, base_prompt, inputs, examples_content):
    responses = []
    response_times = []
    # Initialize the message history with the system message (prompt)
    message_history = [{"role": "system", "content": base_prompt}]

    # Parse and add the examples content to the message history
    examples_data = json.loads(f'{examples_content}')
    for item in examples_data:
        message_history.append({
            "role": item["role"],
            "content": item["content"]
        })

    for user_input in inputs:
        # Add the current user input to the message history
        message_history.append({"role": "user", "content": user_input})

        start_time = time.time()
        try_count = 0
        while try_count < 3:
            try:
                completion = openai.chat.completions.create(
                    model="gpt-4o",
                    messages=message_history,
                    temperature=0,
                    max_tokens=1500,
                    top_p=1,
                    frequency_penalty=0.0,
                    presence_penalty=0
                )
                end_time = time.time()
                response_content = completion.choices[0].message.content
                # Add the assistant's response to the message history
                message_history.append({"role": "assistant", "content": response_content})

                responses.append(response_content)
                response_times.append(end_time - start_time)

                # Print the completion output here
                print(completion)
                print(f"Order {order}, Input: '{user_input}', Response: '{response_content}', Time: {end_time - start_time:.2f}s\n====")
                break
            except OpenAIError:
                try_count += 1
                if try_count >= 3:
                    responses.append("Request failed after 2 retries.")
                    response_times.append("-")
                    print(f"Order {order}, Input: '{user_input}', Response: 'Request failed after 2 retries.', Time: -")
                else:
                    time.sleep(3)  # Wait for 10 seconds before retrying

    # Reset the message history for the next order
    return responses, response_times, message_history

# Load the input Excel file
df_input = pd.read_excel('gogo.xlsx')

# List to store rows before appending them to the DataFrame
output_rows = []

for index, row in df_input.iterrows():
    order = row['order']
    prompt = row['prompt']
    inputs = [row['first']]
    examples_content = row['examples']  # Assuming the column is named 'examples'

    responses, response_times, message_history = process_conversation(order, prompt, inputs, examples_content)

    for i, user_input in enumerate(inputs):
        output_rows.append({
            'Order': order,
            'User Input': user_input,
            'Output Message': responses[i],
            'Response Time': response_times[i]
        })

# Create a DataFrame from the list of output rows
df_output = pd.DataFrame(output_rows, columns=['Order', 'User Input', 'Output Message', 'Response Time'])

# Save the results to an Excel file
df_output.to_excel('output_test.xlsx', index=False)

KeyError: 'examples'

In [4]:
# @title OPENAI KO CÓ MESSAGE HISTORY
def process_conversation(order, base_prompt, inputs):
    responses = []
    response_times = []
    # Initialize the message history with the system message (prompt)
    message_history = [{"role": "system", "content": base_prompt}]


    for user_input in inputs:
        # Add the current user input to the message history
        message_history.append({"role": "user", "content": user_input})

        start_time = time.time()
        try_count = 0
        while try_count < 3:
            try:
                completion = openai.chat.completions.create(
                    model="gpt-4o",
                    messages=message_history,
                    temperature=0,
                    max_tokens=1500,
                    top_p=1,
                    frequency_penalty=0.0,
                    presence_penalty=0.0
                )
                end_time = time.time()
                response_content = completion.choices[0].message.content
                # Add the assistant's response to the message history
                message_history.append({"role": "assistant", "content": response_content})

                responses.append(response_content)
                response_times.append(end_time - start_time)

                # Print the completion output here
                print(f"Order {order}, Input: '{user_input}', Response: '{response_content}', Time: {end_time - start_time:.2f}s\n====")
                break
            except OpenAIError:
                try_count += 1
                if try_count >= 3:
                    responses.append("Request failed after 2 retries.")
                    response_times.append("-")
                    print(f"Order {order}, Input: '{user_input}', Response: 'Request failed after 2 retries.', Time: -")
                else:
                    time.sleep(3)  # Wait for 10 seconds before retrying

    # Reset the message history for the next order
    return  responses, response_times, message_history

# Load the input Excel file
df_input = pd.read_excel('check.xlsx')

# List to store rows before appending them to the DataFrame
output_rows = []

for index, row in df_input.iterrows():
    order = row['order']
    prompt = row['prompt']
    inputs = [row['first']]

    responses, response_times, message_history = process_conversation(order, prompt, inputs)

    for i, user_input in enumerate(inputs):
        output_rows.append({
            'Order': order,
            'User Input': user_input,
            'Output Message': responses[i],
            'Response Time': response_times[i]
        })

# Create a DataFrame from the list of output rows
df_output = pd.DataFrame(output_rows, columns=['Order', 'User Input', 'Output Message', 'Response Time'])

# Save the results to an Excel file
df_output.to_excel('output_1.xlsx', index=False)


Order 1, Input: 'question: In the business world, being too friendly can sometimes be perceived as a weakness. It's important to find a balance between being approachable and maintaining professionalism. What are you looking for in a job? \nanswer: I am looking for a job that offers opportunities for growth and development, a positive work environment, and a good work-life balance.', Response: '{"check_grammar": true, "correct_explain": true, "correct_answer_fix": true}', Time: 0.76s
====
Order 2, Input: 'question: It's great that you have such clear goals in mind. I admire your purpose. How long have you been learning English? \nanswer: I have been learning English for 10 years.', Response: '{"check_grammar": true, "correct_explain": true, "correct_answer_fix": true}', Time: 2.30s
====
Order 3, Input: 'question: I see. It's great that you were able to overcome that challenge. I think I might face a similar situation. How did you handle/overcome that challenge? And also, what skills do

In [ ]:
# @title LLAMA 3 CÓ MESSAGE HISTORY
def process_conversation(order, base_prompt, inputs, examples_content):
    responses = []
    response_times = []
    # Initialize the message history with the system message (prompt)
    message_history = [{"role": "system", "content": base_prompt}]

    # Parse and add the examples content to the message history
    examples_data = json.loads(f'[{examples_content}]')
    for item in examples_data:
        message_history.append({
            "role": item["role"],
            "content": item["content"]
        })

    for user_input in inputs:
        # Add the current user input to the message history
        message_history.append({"role": "user", "content": user_input})

        start_time = time.time()
        try_count = 0
        while try_count < 3:
            try:
                completion = client.chat.completions.create(
                    model="meta-llama/Llama-3-70b-chat-hf",
                    messages=message_history,
                    temperature=0,
                    max_tokens=2500,
                    top_p=1,
                    frequency_penalty=0.0,
                    presence_penalty=0
                )
                end_time = time.time()
                response_content = completion.choices[0].message.content
                # Add the assistant's response to the message history
                message_history.append({"role": "assistant", "content": response_content})

                responses.append(response_content)
                response_times.append(end_time - start_time)

                # Print the completion output here
                print(f"Order {order}, Input: '{user_input}', Response: '{response_content}', Time: {end_time - start_time:.2f}s\n====")
                break
            except OpenAIError:
                try_count += 1
                if try_count >= 3:
                    responses.append("Request failed after 2 retries.")
                    response_times.append("-")
                    print(f"Order {order}, Input: '{user_input}', Response: 'Request failed after 2 retries.', Time: -")
                else:
                    time.sleep(10)  # Wait for 10 seconds before retrying

    # Reset the message history for the next order
    return responses, response_times, message_history

# Load the input Excel file
df_input = pd.read_excel('input_with_example_goc.xlsx')

# List to store rows before appending them to the DataFrame
output_rows = []

for index, row in df_input.iterrows():
    order = row['order']
    prompt = row['prompt']
    inputs = [row['first']]
    examples_content = row['examples']  # Assuming the column is named 'examples'

    responses, response_times, message_history = process_conversation(order, prompt, inputs, examples_content)

    for i, user_input in enumerate(inputs):
        output_rows.append({
            'Order': order,
            'User Input': user_input,
            'Output Message': responses[i],
            'Response Time': response_times[i]
        })

# Create a DataFrame from the list of output rows
df_output = pd.DataFrame(output_rows, columns=['Order', 'User Input', 'Output Message', 'Response Time'])

# Save the results to an Excel file
df_output.to_excel('input_with_example_goc_llma3.xlsx', index=False)


In [ ]:
# @title LLAMA3 KO CÓ MESSAGE HISTORY
def process_conversation(order, base_prompt, inputs):
    responses = []
    response_times = []
    # Initialize the message history with the system message (prompt)
    message_history = [{"role": "system", "content": base_prompt}]


    for user_input in inputs:
        # Add the current user input to the message history
        message_history.append({"role": "user", "content": user_input})

        start_time = time.time()
        try_count = 0
        while try_count < 3:
            try:
                completion = client.chat.completions.create(
                    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
                    messages=message_history,
                    temperature=0,
                    max_tokens=2500,
                    top_p=0.5,
                    frequency_penalty=0.0,
                    presence_penalty=0
                )
                end_time = time.time()
                response_content = completion.choices[0].message.content
                # Add the assistant's response to the message history
                message_history.append({"role": "assistant", "content": response_content})

                responses.append(response_content)
                response_times.append(end_time - start_time)

                # Print the completion output here
                print(f"Order {order}, Input: '{user_input}', Response: '{response_content}', Time: {end_time - start_time:.2f}s\n====")
                break
            except OpenAIError:
                try_count += 1
                if try_count >= 3:
                    responses.append("Request failed after 2 retries.")
                    response_times.append("-")
                    print(f"Order {order}, Input: '{user_input}', Response: 'Request failed after 2 retries.', Time: -")
                else:
                    time.sleep(10)  # Wait for 10 seconds before retrying

    # Reset the message history for the next order
    return responses, response_times, message_history

# Load the input Excel file
df_input = pd.read_excel('omg.xlsx')

# List to store rows before appending them to the DataFrame
output_rows = []

for index, row in df_input.iterrows():
    order = row['order']
    prompt = row['prompt']
    inputs = [row['first']]


    responses, response_times, message_history = process_conversation(order, prompt, inputs)

    for i, user_input in enumerate(inputs):
        output_rows.append({
            'Order': order,
            'User Input': user_input,
            'Output Message': responses[i],
            'Response Time': response_times[i]
        })

# Create a DataFrame from the list of output rows
df_output = pd.DataFrame(output_rows, columns=['Order', 'User Input', 'Output Message', 'Response Time'])

# Save the results to an Excel file
df_output.to_excel('output_CONVER.xlsx', index=False)


Order 1, Input: 'Topic: Hometown - Location and direction
Given vocab: In the North of the country', Response: 'Here is the conversation:

[{"situation":"Hai người bạn, Alex và Maddie, đang đi du lịch đến quê hương của Alex ở Hà Nội","situation_en":"Two friends, Alex and Maddie, are traveling to Alex's hometown in Hanoi","conversation":[{"role":"male_1","content":"Maddie ơi, tôi rất hào hứng đưa bạn đến quê hương của tôi ở Hà Nội","content_en":"Maddie, I'm so excited to bring you to my hometown in Hanoi"},{"role":"female_1","content":"Tôi cũng rất hào hứng, Alex! Tôi đã nghe nói về vẻ đẹp của Hà Nội. Nhưng làm thế nào chúng ta có thể đến đó từ đây?","content_en":"I'm also excited, Alex! I've heard about the beauty of Hanoi. But how can we get there from here?"},{"role":"male_2","content":"À, chúng ta có thể đi xe lửa từ Sài Gòn đến Hà Nội. Hà Nội nằm ở <r>Bắc</r> của đất nước","content_en":"Ah, we can take the train from Saigon to Hanoi. Hanoi is located <r>In the North of the country<

KeyboardInterrupt: 

In [ ]:
# @title OPENAI IELTS SCORE
def process_conversation(order, base_prompt, inputs):
    responses = []
    response_times = []
    message_history = [{"role": "system", "content": base_prompt}]

    for user_input in inputs:
        message_history.append({"role": "user", "content": user_input})

        start_time = time.time()
        try_count = 0
        while try_count < 3:
            try:
                completion = openai.chat.completions.create(
                    model="gpt-4o",
                    messages=message_history,
                    temperature=0,
                    max_tokens=2500,
                    top_p=0,
                    frequency_penalty=0.0,
                    presence_penalty=0
                )
                end_time = time.time()
                response_content = completion.choices[0].message.content
                message_history.append({"role": "assistant", "content": response_content})
                print(response_content)
                responses.append(response_content)
                response_times.append(end_time - start_time)
                print(f"Order {order}, Input: '{user_input}', Response: '{response_content}', Time: {end_time - start_time:.2f}s\n====")
                break
            except OpenAIError as e:
                try_count += 1
                if try_count >= 3:
                    responses.append("Request failed after 2 retries.")
                    response_times.append("-")
                    print(f"Order {order}, Input: '{user_input}', Response: 'Request failed after 2 retries.', Time: -")
                else:
                    time.sleep(10)  # Wait for 10 seconds before retrying

    return responses, response_times, message_history

def process_responses(responses, columns):
    last_yes_response = None
    for response, column in zip(responses, columns):
        if "[match]: Yes" in response:
            last_yes_response = (response, column)
        elif "[match]: No" in response:
            break
    return last_yes_response

for index, row in df_input.iterrows():
    order = row['order']
    columns_to_check = ['c', 'd', 'e', 'f'] if "[match]: Yes" in process_conversation(order, row['c'], [row['presentation']])[0][0] else ['c', 'b', 'a']
    responses, _, _ = zip(*[process_conversation(order, row[col], [row['presentation']]) for col in columns_to_check])
    last_yes_response = process_responses(responses, columns_to_check)
    if last_yes_response:
        response, band = last_yes_response
        comment = response.split('[analytic]:')[1].split('[match]:')[0].strip()
        output_rows.append({
            'Order': order,
            'Comment': comment,
            'Band': band
        })

df_output = pd.DataFrame(output_rows, columns=['Order', 'Comment', 'Band'])
df_output.to_excel('output_file.xlsx', index=False)

[analytic]:
- **Resource sufficient to discuss topics at length**: The candidate demonstrates the ability to discuss various topics at length, including their studies, work, hometown, hobbies, and financial habits.
- **Vocabulary use may be inappropriate but meaning is clear**: There are minor instances of awkward phrasing (e.g., "I got it totally fixed" instead of "I had it repaired"), but the meaning remains clear throughout the conversation.
- **Generally able to paraphrase successfully**: The candidate shows the ability to paraphrase and explain their points in different ways, such as describing their saving habits and the reasons for carrying cash in winter.

[match]: Yes
Order 1, Input: '[Fluency]: 
- speech rate: 112 wpm 
- Pausing score: 80.77 (higher better)
====
[examiner]: Alright, let's begin. So, could you tell me your full name, please?
[candidate]: Hi, my name is Yoonjae Lee, and I go by the name Yoonjae.
[examiner]: Okay. And Yoonjae, are you working or are you a studen

KeyboardInterrupt: 

In [ ]:
df_output = pd.DataFrame(output_rows, columns=['Order', 'Comment', 'Band'])
df_output.to_excel('output_file.xlsx', index=False)

In [ ]:
import requests

# Set the URL for the API endpoint
url = "https://api.elsanow.io/api/v1/score_audio_plus"

# Define the headers and files to be sent
headers = {
    "Authorization": "ELSA DX+gsaRdJgTQN7K3XtShZ5b6x/KnAn7pa7QPf777QVEvkDGywHO/z5H+ughZR9E0bWYXtr116X5LGNodpFIra+s6jJPUmrKaT5wn/Np2ICk="
}
files = {
    "audio": ("filename.wav", open("Untitled Session 1_mixdown.wav", "rb")),
    "api_plan": (None, "premium"),
    "return_json": (None, "true")
}

# Function to extract and parse the scores from the JSON response
def extract_metrics(response_json):
    """
    Extract and parse the specified metrics from the JSON response.
    """
    try:
        speakers = response_json['speakers']
        metrics = speakers[0]['metrics']['general_scores']['elsa']


        pronunciation_score = metrics['pronunciation_score']
        intonation_score = metrics['intonation_score']

        speakers = response_json['speakers']
        metrics = speakers[0]['metrics']['other_metrics']['fluency']

        words_per_minute = metrics['words_per_minute']
        pausing_score = metrics['pausing_score']

        parsed_result = f"""
        [Fluency]:
        - Words per minute: {words_per_minute}
        - Pausing score: {pausing_score}
        [Pronunciation]:
        - Pronunciation score: {pronunciation_score}
        - Intonation score: {intonation_score}
        """
        return parsed_result
    except (KeyError, IndexError) as e:
        return f"Error extracting metrics: {e}"

# Send the POST request
response = requests.post(url, headers=headers, files=files)

# Check if the request was successful
if response.status_code == 200:
    response_json = response.json()
    parsed_metrics = extract_metrics(response_json)
    print(parsed_metrics)
    print(response_json)
else:
    print(f"Failed to get a valid response: {response.status_code}")


        [Fluency]:
        - Words per minute: 114
        - Pausing score: 73.19
        [Pronunciation]:
        - Pronunciation score: 71.04
        - Intonation score: 70.24
        
{'speakers': [{'speaker_id': 0, 'total_time': 248.52000000000007, 'metrics': {'general_scores': {'elsa': {'eps_score': 77.72, 'eps_decision': 'correct', 'pronunciation_score': 71.04, 'pronunciation_decision': 'warning', 'intonation_score': 70.24, 'intonation_decision': 'warning', 'fluency_score': 80.39, 'fluency_decision': 'correct', 'grammar_score': 93.3, 'grammar_decision': 'correct', 'vocabulary_score': 73.64, 'vocabulary_decision': 'warning'}, 'cefr': {'overall_cefr': 'C1', 'pronunciation_cefr': 'B2', 'intonation_cefr': 'B2', 'fluency_cefr': 'C1', 'grammar_cefr': 'C2', 'vocabulary_cefr': 'B2'}, 'other_scores': {'ielts_score': 7.0, 'toefl_score': 23, 'toeic_score': 180, 'pte_score': 67}}, 'other_metrics': {'fluency': {'words_per_minute': 114, 'words_per_minute_min': 62, 'words_per_minute_max': 164,

In [ ]:
import pandas as pd
from pathlib import Path
import openai



# Load the Excel file
excel_file_path = Path("input.xlsx")
df = pd.read_excel(excel_file_path)

# Directory to save MP3 files
output_dir = Path("./")
output_dir.mkdir(parents=True, exist_ok=True)

# Process each line in the Excel file
for index, row in df.iterrows():
    input_text = row['input text']
    speech_file_path = output_dir / f"{index + 1}.mp3"

    # Create the speech
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=input_text
    )

    # Save the speech to the specified file path
    response.stream_to_file(speech_file_path)


<ipython-input-3-7bf89d64615f>:28: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)
